In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
import string
import zeyrek
from TurkishStemmer import TurkishStemmer

%matplotlib inline

In [2]:
# Importing Data Set
df = pd.read_csv("/Users/ideakadikoy/Desktop/kadıkoy_df.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20242 entries, 0 to 20241
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Entry       20242 non-null  object
 1   Author      20242 non-null  object
 2   Date        20242 non-null  object
 3   Old_Topic   20242 non-null  object
 4   New_Topic   20242 non-null  object
 5   Edited      20242 non-null  bool  
 6   Entry_Date  20242 non-null  object
 7   Edit_Date   3620 non-null   object
dtypes: bool(1), object(7)
memory usage: 1.1+ MB


In [6]:
# Turkish Stop Words
trstop = [
    'a', 'acaba', 'altı', 'altmış', 'ama', 'ancak', 'arada', 'artık', 'asla', 'aslında', 'aslında', 'ayrıca', 'az', 'bana',
    'bazen', 'bazı', 'bazıları', 'belki', 'ben', 'benden', 'beni', 'benim', 'beri', 'beş', 'bile', 'bilhassa', 'bin', 'bir',
    'biraz', 'birçoğu', 'birçok', 'biri', 'birisi', 'birkaç', 'birşey', 'biz', 'bizden', 'bize', 'bizi', 'bizim', 'böyle',
    'böylece', 'bu', 'buna', 'bunda', 'bundan', 'bunlar', 'bunları', 'bunların', 'bunu', 'bunun', 'burada', 'bütün', 'çoğu',
    'çoğunu', 'çok', 'çünkü', 'da', 'daha', 'dahi', 'dan', 'de', 'defa', 'değil', 'diğer', 'diğeri', 'diğerleri', 'diye',
    'doksan', 'dokuz', 'dolayı', 'dolayısıyla', 'dört', 'e', 'edecek', 'eden', 'ederek', 'edilecek', 'ediliyor', 'edilmesi',
    'ediyor', 'eğer', 'elbette', 'elli', 'en', 'etmesi', 'etti', 'ettiği', 'ettiğini', 'fakat', 'falan', 'filan', 'gene',
    'gereği', 'gerek', 'gibi', 'göre', 'hala', 'halde', 'halen', 'hangi', 'hangisi', 'hani', 'hatta', 'hem', 'henüz', 'hep',
    'hepsi', 'her', 'herhangi', 'herkes', 'herkese', 'herkesi', 'herkesin', 'hiç', 'hiçbir', 'hiçbiri', 'i', 'ı', 'için',
    'içinde', 'iki', 'ile', 'ilgili', 'ise', 'işte', 'itibaren', 'itibariyle', 'kaç', 'kadar', 'karşın', 'kendi', 'kendilerine',
    'kendine', 'kendini', 'kendisi', 'kendisine', 'kendisini', 'kez', 'ki', 'kim', 'kime', 'kimi', 'kimin', 'kimisi', 'kimse',
    'kırk', 'madem', 'mi', 'mı', 'milyar', 'milyon', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nedenle', 'nerde', 'nerede', 'nereye',
    'neyse', 'niçin', 'nin', 'nın', 'niye', 'nun', 'nün', 'o', 'öbür', 'olan', 'olarak', 'oldu', 'olduğu', 'olduğunu',
    'olduklarını', 'olmadı', 'olmadığı', 'olmak', 'olması', 'olmayan', 'olmaz', 'olsa', 'olsun', 'olup', 'olur', 'olur',
    'olursa', 'oluyor', 'on', 'ön', 'ona', 'önce', 'ondan', 'onlar', 'onlara', 'onlardan', 'onları', 'onların', 'onu', 'onun',
    'orada', 'öte', 'ötürü', 'otuz', 'öyle', 'oysa', 'pek', 'rağmen', 'sana', 'sanki', 'sanki', 'şayet', 'şekilde', 'sekiz',
    'seksen', 'sen', 'senden', 'seni', 'senin', 'şey', 'şeyden', 'şeye', 'şeyi', 'şeyler', 'şimdi', 'siz', 'siz', 'sizden',
    'sizden', 'size', 'sizi', 'sizi', 'sizin', 'sizin', 'sonra', 'şöyle', 'şu', 'şuna', 'şunları', 'şunu', 'ta', 'tabii',
    'tam', 'tamam', 'tamamen', 'tarafından', 'trilyon', 'tüm', 'tümü', 'u', 'ü', 'üç', 'un', 'ün', 'üzere', 'var', 'vardı',
    've', 'veya', 'ya', 'yani', 'yapacak', 'yapılan', 'yapılması', 'yapıyor', 'yapmak', 'yaptı', 'yaptığı', 'yaptığını',
    'yaptıkları', 'ye', 'yedi', 'yerine', 'yetmiş', 'yi', 'yı', 'yine', 'yirmi', 'yoksa', 'yu', 'yüz', 'zaten', 'zira'
] # https://github.com/ahmetax/trstop/blob/master/dosyalar/turkce-stop-words

nltk_trstop = [
    'acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha',
    'de', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim',
    'mı', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey', 'siz', 'şu',
    'tüm', 've', 'veya', 'ya', 'yani'
] # https://github.com/xiamx/node-nltk-stopwords/blob/master/data/stopwords/turkish

add_stop = [
    "sayın", "başkan", "milletvekili", "sıralarından", "değerli", "alkışlar", "ilişkin", "devamla", "milletvekilleri",
    "devam", "teşekkür", "arkadaşlar", "adına", "açıklaması", "buyurun", "mikrofon", "ediyorum", "grubu", "saygıyla",
    "selamlıyorum", "bakın", "zaman", "karşı", "sadece", "önemli", "üzerinde", "sayılı", "ifade", "başkanım", "diyor",
    "ederim", "istiyorum", "birlikte", "sunuyorum", "özellikle", "başka", "içerisinde", "üzerine", "etmek", "buyurunuz",
    "efendim", "evet", "hayır", "ortaya", "yönelik", "vardır", "edilmiştir", "gelen", "maalesef", "arasında", "lütfen",
    "hakkında", "altında", "buradan", "kesin", "emin", "uygun", "ediyoruz", "lazım", "edilen", "gerçekten", "peki",
    "konuda", "amacıyla", "gereken", "yapılmasına", "değildir", "bulunan", "başta", "tekrar", "dedi", "almış", "olacak",
    "konusunda", "açısından", "yapan", "gerekiyor", "musunuz", "mümkün", "s", "itibarıyla", "demek", "gerekli",
    "hepinizi", "söylüyorum", "diliyorum", "veren", "ederiz", "etmeyenler", "sorun", "yoktur", "görüşülmekte",
    "alınması", "yanında", "geliyor", "edilmemiştir", "diyorum", "varsa", "alınan", "biliyoruz", "teşekkürler", "aldım",
    "aldı", "alınmış", "alınmıştır","bkz", "yok", "sadece", "aynı", "büyük", "edit", "gerçekten", "in", "ın", "bence", "olmuş", "amk", "den", "demek",
    "geliyor", "gelen", "umarım", "vardır", "sanırım", "mesela", "düşünüyorum", "lan", "vs",'a', 'b', 'c', 'ç', 'd', 'e', 'f', 'g', 'ğ', 'h', 'ı', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'ö', 'p', 'r', 's', 'ş', 't',
    'u', 'ü', 'v', 'y', 'z'
]

stop_words = sorted(list(set(trstop).union(nltk_trstop).union(add_stop)))

In [7]:
analyzer = zeyrek.MorphAnalyzer()

def lemmatizer(sentence):
    return ' '.join([item[1][0].lower() for item in analyzer.lemmatize(sentence)])

In [8]:
stem = TurkishStemmer()

def stemmer(sentence):
    return ' '.join([stem.stem(word) for word in sentence.split()])

non_stem_words = ["Kadıköy", "Kadıköy belediyesi", "moda", "sahil", "haydarpaşa", "caferaga", "suadiye", "erenköy",
                  "acıbadem", "bostancı", "caddebostan","yeldeğirmeni","barlar"]

def stemmer(sentence):
    return ' '.join([stem.stem(word) if word not in non_stem_words else word for word in sentence.split()])

In [9]:
# Remove urls
df["Processed_Entry"] = df["Entry"].apply(lambda x: ' '.join([word for word in x.split() if word[:4] != "http"]))
df["Processed_Entry"] = df["Processed_Entry"].apply(lambda x: ' '.join([word for word in x.split() if word[:3] != "www"]))
df["Processed_Entry"] = df["Processed_Entry"].apply(lambda x: ' '.join([word for word in x.split() if word != "via"]))
df["Processed_Entry"] = df["Processed_Entry"].apply(lambda x: ' '.join([word for word in x.split() if word != "aracılığıyla"]))
# Remove punctuation --> '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
df["Processed_Entry"] = df["Processed_Entry"].apply(lambda x: ''.join([character if character not in string.punctuation else ' ' for character in x]))
# Remove stop words
df["Processed_Entry"] = df["Processed_Entry"].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
# Remove numeric characters
df["Processed_Entry"] = df["Processed_Entry"].apply(lambda x: ' '.join([word for word in x.split() if word.isalpha()]))
# Remove single letters
df["Processed_Entry"] = df["Processed_Entry"].apply(lambda x: ' '.join([word for word in x.split() if len(word) > 1]))

In [14]:
df["Processed_Entry"]

0        arife gunu yarim gun mesaisini yapmami tembel ...
1        sorumluluklarin getirse kadikoy sevecen anabil...
2        sağa sola kent yaşamının gerçek sahibi yayalar...
3                                kadikoy afet kriz merkezi
4        internet sitesinde sahipsiz kedi köpekler bölü...
                               ...                        
20237      türkiye istanbul ankara izmir saçma sapan tanım
20238    bostancı nesi sorarsanız güzergahta bostancı s...
20239    göztepe caddebostan dahil istanbul hoş semtler...
20240    çocukluğum ergenliğim gençliğim kısacası hayat...
20241    istanbul mülteci turist istilasına uğramayan s...
Name: Processed_Entry, Length: 20242, dtype: object

In [15]:
# Stemming
print("Stemming progress:")
for i in range(len(df)):
    df.loc[i, "Stemmed_Entry"] = stemmer(df.loc[i, "Processed_Entry"])
    print(f"{i+1}/{len(df)} | {(i+1)/len(df)*100:.2f}%")
    
# Remove stop words again
df["Stemmed_Entry"] = df["Stemmed_Entry"].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

Stemming progress:
1/20242 | 0.00%
2/20242 | 0.01%
3/20242 | 0.01%
4/20242 | 0.02%
5/20242 | 0.02%
6/20242 | 0.03%
7/20242 | 0.03%
8/20242 | 0.04%
9/20242 | 0.04%
10/20242 | 0.05%
11/20242 | 0.05%
12/20242 | 0.06%
13/20242 | 0.06%
14/20242 | 0.07%
15/20242 | 0.07%
16/20242 | 0.08%
17/20242 | 0.08%
18/20242 | 0.09%
19/20242 | 0.09%
20/20242 | 0.10%
21/20242 | 0.10%
22/20242 | 0.11%
23/20242 | 0.11%
24/20242 | 0.12%
25/20242 | 0.12%
26/20242 | 0.13%
27/20242 | 0.13%
28/20242 | 0.14%
29/20242 | 0.14%
30/20242 | 0.15%
31/20242 | 0.15%
32/20242 | 0.16%
33/20242 | 0.16%
34/20242 | 0.17%
35/20242 | 0.17%
36/20242 | 0.18%
37/20242 | 0.18%
38/20242 | 0.19%
39/20242 | 0.19%
40/20242 | 0.20%
41/20242 | 0.20%
42/20242 | 0.21%
43/20242 | 0.21%
44/20242 | 0.22%
45/20242 | 0.22%
46/20242 | 0.23%
47/20242 | 0.23%
48/20242 | 0.24%
49/20242 | 0.24%
50/20242 | 0.25%
51/20242 | 0.25%
52/20242 | 0.26%
53/20242 | 0.26%
54/20242 | 0.27%
55/20242 | 0.27%
56/20242 | 0.28%
57/20242 | 0.28%
58/20242 | 0.29%
59/2

In [88]:
df.to_csv("entries_processed.csv", index=False)